# Instructions for Building the Restaurant Map Using Dash and Flask

### Objectives:

-  Setup virtual environment for Flask/Dash app
-  Write our first route and link it to a simple html index page
-  Connect a dash server to live on our Flask app
-  Create line, scatter, and choropleth visualizations on the Dashboard

## Installation

To get started, we will need a fresh virtual environment. Hopefully you have already run the following installs, as they may take some time: 

```$ conda install virtualenv
$ virtualenv plotlybox
$ source plotlybox/bin/activate```

Once you have the virtualenv running (you should no longer see (dsi) but instead (plotlybox) next to your username), we need to do some installations to make sure everything will work. 

`$ pip install Flask pandas scikit-learn scipy plotly dash dash-html-components dash-core-components dash-table pandas_datareader flask_cors datetime`

## Setting up your first webserver (Flask)

Okay, now we are ready to build some code. Let's start with a simple flask app. 

```$ touch dashboard.py
$ open dashboard.py```

Note: I have my atom linked to sublime text. Your open command may choose another text editor or Integrated Development Environment. As long as it gives you the ability to edit text and then save it, you should be good.  

The Flask website instructs us to copy and paste the following code into our now open file, save it, then run it with a special Flask command. Let's follow their directions to get started. 

```
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"
    ```

Paste that code into your open dashboard.py, then run the following code below: 

``` 
$ export FLASK_APP=dashboard.py 
$ flask run
```

At this point, if you navigate to http://127.0.0.1:5000/ you should see a "Hello World" greeting. 

Great! Let's see where we can go with this. 

## Adding html templates

Flask works by running the 'backend' logic code from the .py file, but it doesn't want to worry about html, css, and javascript in the same file. In order to compartimentalize these concerns, we are going to add a /templates directory to our project file. 

```
$ mkdir templates
$ touch templates/index.html
$ open templates/index.html```

You should now be looking at a blank html file. Here you go: 

```
<!DOCTYPE html>
<html lang="en" dir="ltr">
  <head>
    <meta charset="utf-8">
    <title></title>
  </head>
  <body>
    <h2>Welcome to Dashboard Visualization!</h2>

    <p>This project is a simple display of what is possible with flask by integrating another package called Dash.</p>

    <p><a href="/visualization">Visualization</a></p>

  </body>
</html>
```



## Flask w/ Dash

Okay, cool, now we have a standalone flask page with some dead links, and a separate dash server that does some neat visualizations. What we would like is if we could link to the visualization in Dash from the homepage in flask. 

Well, notice the similar structure in these two files: 

Flask needs a server variable, that gets assigned with the following line of code: 

`app = Flask(__name__)`

In the same way, Dash (which is actually running on top of flask), needs a server. 

`app = dash.Dash()`

We are going to modify these two lines of code so that the dash app uses the flask app we setup, instead of running its own (default behavior). 

```
app = Flask(__name__)
dapp = dash.Dash(__name__, server=app, url_base_pathname='/visualization/')
```

In the flask file, add the dapp line to get the dash app. We see the server variable to `app`, our flask server, and the url_base_pathname to `/visualization/` so that dash knows where to run itself. 

Then we can copy and paste our dash app over into our flask server with just a few changes and it will run. See the finished code below: 

```
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt
import pandas as pd

from flask import Flask, render_template
app = Flask(__name__)
dapp = dash.Dash(__name__, server=app, url_base_pathname='/visualization/')

@app.route("/")
@app.route("/index")
def main_page():
    return render_template('index.html')

mapbox_access_token = 'pk.eyJ1IjoibmV1cmFsYWxjaGVtaXN0IiwiYSI6ImNqcWZ0Z3BuOTF4aXo0M3FwdGQ0YnllZXoifQ.xqpGQfUtBOXhKU_SYN4l5w'

map_data = pd.read_csv("./df_dash.csv")
map_data = map_data[["name", "price", "location", "latitude", "longitude", "totalvalue","totalvalue_bin"]].drop_duplicates()

dapp.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'})

dapp.title = "LA Restaurant Map"

dapp.layout = html.Div([ html.H2('LA Property Value Mapping'),
    html.Div([
    html.Div([
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 1, 'value': 1},
            {'label': 2, 'value': 2},
            {'label': 3, 'value': 3},
            {'label': 4, 'value': 4}
        ],
        value='SPY'
    )
    # dcc.Graph(id='my-graph')
    ], style = {'flex':'49%'}),
    html.Div([
    dcc.Dropdown(
        id='dropdown-2',
        options=[
            {'label': '< $400K', 'value': '< $400K'},
            {'label': '$400-$800K', 'value': '$400-$800K'},
            {'label': '$800-$1,200K' , 'value': '$800-$1,200K'},
            {'label': '> $1,200K', 'value': '> $1,200K'}
        ],
        value='GOOG'
    )
    # dcc.Graph(id='scatter')
    ], style = {'flex':'49%'})
], style = {'display':'flex'}),
    html.Div([
                html.Div(
                    [
                        dcc.Graph(id='map-graph',
                                  animate=True,
                                  style={'margin-top': '20'},
                                  figure={"data": [{"type": "scattermapbox",
                                                    "lat": list(map_data['latitude']),
                                                    "lon": list(map_data['longitude']),
                                                    "hoverinfo": "text",
                                                    "hovertext": [["Name: {} <br>Price: {} <br>Zipcode: {}".format(i,j,k)]
                                                                    for i,j,k in zip(map_data['name'], map_data['price'],map_data['location'])],
                                                    "mode": "markers",
                                                    "name": list(map_data['name']),
                                                    "marker": {
                                                        "size": 6,
                                                        "color" : map_data['totalvalue'],
                                                        "colorscale" : 'Viridis',
                                                        "opacity": 0.7
                                                    }
                                            }],
                                            "layout": dict(autosize=True,
                                                            height=500,
                                                            font=dict(color="#191A1A"),
                                                            titlefont=dict(color="#191A1A", size='14'),
                                                            margin=dict(
                                                                l=35,
                                                                r=35,
                                                                b=35,
                                                                t=45
                                                            ),
                                                            hovermode="closest",
                                                            plot_bgcolor='#fffcfc',
                                                            paper_bgcolor='#fffcfc',
                                                            legend=dict(font=dict(size=10), orientation='h'),
                                                            title='LA Restaurant Locations (via Yelp API)',
                                                            mapbox=dict(
                                                                accesstoken=mapbox_access_token,
                                                                style="light",
                                                                center=dict(
                                                                    lon=-118.369483,
                                                                    lat=34.081971
                                                                ),
                                                                zoom=10,
                                                            )
                                                        )
                                            }),
                        html.Div([html.A('DSI Dash Demo', href = 'https://github.com/amyoshino/Dash_Tutorial_Series/blob/master/ex4.py')],
                            style={'text-align':'center'})
                    ],
                ),


            ], className="row"
        )
    ], className='ten columns offset-by-one')
    ```

Now you can `$ flask run` one last time and you will see the html page we built before, only this time when you click on the visualizations link you will be redirected to your dashboard. Congrats! You are well on your way to being able to build professional looking data science dashboards. 

# Further Resources

Here are some links that will be helpful: 

- Dash documentation: https://dash.plot.ly/  
- Plot.ly documentation: https://plot.ly/python/  
- Plot.ly Jupyter notebook tutorial: https://plot.ly/python/ipython-notebook-tutorial/
- Dash examples: https://dash.plot.ly/gallery  

# Sources

DSI 6 Lecture Notes - Chris Sanatra - LA local TA